# Retention Time Preidiction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This is an extension of the original walkthrough example available [here](https://github.com/wilhelm-lab/dlomix-resources/blob/main/notebooks/RetentionTime/Example_RTModel_Walkthrough_colab.ipynb).

### Task 2: Loss Function
Similar to the initial notebook, we will initialize our model and train it. The target here is to experiment with different loss functions and observe thg performance of the trained model. The loss function is our optimization objective, which we use to quantify how good or bad our model, being trained, is performing and find better set of parameters that result in better performance on the task at hand.

In [1]:
# install the mlomix package in the current environment using pip

!python -m pip install -q dlomix==0.0.4

In [ ]:
!python -m pip install -q wandb

In [3]:
import numpy as np
import pandas as pd
import dlomix
from dlomix.models import RetentionTimePredictor
import tensorflow as tf
from dlomix.eval import TimeDeltaMetric

import wandb
from wandb.keras import WandbCallback

In [ ]:
# enter project name for weights and biases
project_name = 'dlomix_retention_time'

In [4]:
from dlomix.data import RetentionTimeDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_train_val.csv'
BATCH_SIZE = 64

rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length=30, batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

The code below creates a model and trains it. You should try out different loss functions and observe the impact on the training. Please Refer to the initial notebook to analyze the results.

Hint: Explore the difference between mean squared error ('mse') and mean absolute error ('mae'). Details about loss functions available in `tf.keras` are here: https://www.tensorflow.org/api_docs/python/tf/keras/losses

In [ ]:
# Enter weights and biases run name. Make sure that different loss functions have different run names.
wandb.init(project=project_name, name='loss_function')

loss_function = 'mse'

# create model
model = RetentionTimePredictor(seq_length=30)

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom time-delta metric

model.compile(optimizer=optimizer, 
            loss=loss_function, metrics=[TimeDeltaMetric()])

history = model.fit(rtdata.train_data, validation_data=rtdata.val_data, epochs=15,
                   callbacks=[WandbCallback(save_model=False)])

# Mark the run as finished
wandb.finish()